<a href="https://colab.research.google.com/github/4L3M4R/intraday/blob/main/indradia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pandas_ta yfinance vaderSentiment transformers torch feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=7619befd2e06bcf545da406fac5ff885e23aca82a467d12ab5824785d925913a
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
import requests
import pandas as pd
#import pandas_ta as ta
from ta.momentum import RSIIndicator, ROCIndicator
from ta.trend import MACD
from ta.volatility import AverageTrueRange, BollingerBands
# import pandas_ta as pta  # Uncomment if you want to use pandas_ta in the future


import yfinance as yf
from datetime import datetime, timedelta
import os


In [ ]:
# --- Leer lista de activos y fuentes ---
with open("activos.txt", "r") as f:
    activos = {}
    for line in f:
        if ":" in line:
            symbol, source = line.strip().split(":")
            activos[symbol.strip()] = source.strip().lower()

# --- Leer configuración ---
config = {}
with open("config.txt", "r") as f:
    for line in f:
        if ":" in line:
            key, value = line.strip().split(":")
            config[key.strip()] = value.strip()

granularity = config.get("frecuencia", "1h")
limit = int(config.get("limit", "100"))
period = config.get("period", "5d")


In [ ]:

# --- Descargar desde Bitget ---
def descargar_datos_bitget(symbol, granularity, limit):
    url = "https://api.bitget.com/api/v2/mix/market/history-candles"
    params = {
        "symbol": symbol,
        "productType": "USDT-FUTURES",
        "granularity": granularity,
        "limit": limit
    }
    print(f"Descargando desde Bitget: {symbol}...")
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json().get("data", [])
        if not data:
            print(f"No hay datos para {symbol}")
            return None
        df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "quoteVolume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"].astype(int), unit='ms', utc=True)
        df["symbol"] = symbol
        return df.sort_values("timestamp")
    else:
        print(f"Error Bitget {symbol}: {response.text}")
        return None

# --- Descargar desde Yahoo Finance ---
def descargar_datos_yf(symbol, interval, period, limit):
    print(f"Descargando desde Yahoo Finance: {symbol}...")
    try:
        data = yf.download(tickers=symbol, interval=interval, period=period)
        if data.empty:
            print(f"No se encontraron datos para {symbol}")
            return None



        # Si las columnas tienen múltiples niveles (multiindex), aplánalas
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [col[0].lower() for col in data.columns]

        else:
            data.columns = [col.lower() for col in data.columns]

        # Reset index para convertir fecha a columna
        data = data.reset_index()

        # Renombrar columnas importantes
        rename_map = {
            'open': 'open',
            'high': 'high',
            'low': 'low',
            'close': 'close',
            'volume': 'volume',
            'Date': 'timestamp'
        }

        data.rename(columns=rename_map, inplace=True)

        # Añadir columna symbol
        data["symbol"] = symbol


        return data.head(limit)
    except Exception as e:
        print(f"Error Yahoo {symbol}: {e}")
        return None


In [ ]:


# --- Cálculo de indicadores ---
def calcular_features(df):
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Price/volume features
    df['gap_apertura_pct'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1) * 100
    df['volatilidad_diaria'] = (df['high'] - df['low']) / df['low'] * 100
    df['vol_5d'] = df['volatilidad_diaria'].rolling(5).std()
    df['vol_10d'] = df['volatilidad_diaria'].rolling(10).std()
    df['vol_rel_5d'] = df['volume'] / df['volume'].rolling(5).mean()
    df['return_pct'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1) * 100
    df['close_open_pct'] = (df['close'] - df['open']) / df['open'] * 100
    df['range_pct'] = (df['high'] - df['low']) / df['low'] * 100
    df['volume_change_pct'] = (df['volume'] - df['volume'].shift(1)) / df['volume'].shift(1) * 100

    # TA indicators using `ta` library
    df['rsi_14'] = RSIIndicator(close=df['close'], window=14).rsi()
    # df['rsi_14'] = pta.rsi(df['close'], length=14)  # pandas_ta version

    macd = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_diff'] = macd.macd_diff()
    # macd = pta.macd(df['close'])
    # df['macd'] = macd['MACD_12_26_9']
    # df['macd_signal'] = macd['MACDs_12_26_9']
    # df['macd_diff'] = df['macd'] - df['macd_signal']

    df['atr_14'] = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14).average_true_range()
    # df['atr_14'] = pta.atr(df['high'], df['low'], df['close'], length=14)

    df['momentum_12'] = ROCIndicator(close=df['close'], window=12).roc()
    # df['momentum_12'] = pta.mom(df['close'], length=12)

    bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_upper'] = bb.bollinger_hband()
    df['bb_lower'] = bb.bollinger_lband()
    df['bb_pctb']  = bb.bollinger_pband()
    # try:
    #     bb = pta.bbands(df['close'], length=20, std=2)
    #     df['bb_upper'] = bb['BBU_20_2.0_2.0']
    #     df['bb_lower'] = bb['BBL_20_2.0_2.0']
    #     df['bb_pctb']  = bb['BBP_20_2.0_2.0']
    # except:
    #     df['bb_upper'] = df['bb_lower'] = df['bb_pctb'] = float('nan')

    df.fillna(0, inplace=True)
    return df


In [ ]:
# --- Guardar ---
def save_df(df, symbol):
    filename = f"{symbol}_datos.txt"
    df.to_csv(filename, sep="\t", index=False)
    print(f"Guardado en {filename}")


In [ ]:
import yfinance as yf
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
import requests
import feedparser
from datetime import datetime

# --- Inicializar modelos ---
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

def calcular_sentiment_vader(text):
    if not text:
        return 0.0
    score = vader_analyzer.polarity_scores(text)
    return score['compound']

def calcular_sentiment_finbert(text):
    if not text:
        return 0.0
    inputs = finbert_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    with torch.no_grad():
        outputs = finbert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1).numpy()[0]
    sentiment_score = scores[0] - scores[2]  # Pos - Neg
    return float(sentiment_score)


# --- Noticias para acciones desde Yahoo Finance ---
def get_news_dataframe(ticker, max_news=50):
    stock = yf.Ticker(ticker)
    try:
        news_items = stock.news[:max_news]
    except Exception:
        return pd.DataFrame()

    data = []
    for item in news_items:
        try:
            content = item.get('content', {})
            title = content.get('title', '').strip()
            description = content.get('description', '').strip()
            summary = content.get('summary', '').strip()
            pubDate = content.get('pubDate')
            if title and pubDate:
                dt = pd.to_datetime(pubDate, utc=True)
                data.append({
                    "title": title,
                    "description": description or summary,
                    "datetime": dt,
                    "date": dt.date()
                })
        except Exception as e:
            print(f"⚠️ Error al procesar noticia: {e}")
            continue
    return pd.DataFrame(data)


# --- Noticias para criptos desde CoinTelegraph RSS ---
def get_crypto_news_cointelegraph(symbol, max_news=50):
    symbol_map = {
        "BTCUSDT": "bitcoin",
        "ETHUSDT": "ethereum",
    }
    tag = symbol_map.get(symbol.upper())
    if not tag:
        print(f"⚠️ No se encontró tag para {symbol}")
        return pd.DataFrame()

    url = f"https://cointelegraph.com/rss/tag/{tag}"
    feed = feedparser.parse(url)
    news_list = []
    for entry in feed.entries[:max_news]:
        title = entry.get("title", "")
        description = entry.get("summary", "")
        published = entry.get("published", "")
        try:
            dt = pd.to_datetime(published, utc=True)
        except:
            continue
        news_list.append({
            "title": title,
            "description": description,
            "datetime": dt,
            "date": dt.date()
        })
    return pd.DataFrame(news_list)

def agregar_sentiment_diario(df, symbol, source):
    if source == "yfinance":
        news_df = get_news_dataframe(symbol, max_news=100)
    elif source == "bitget":
        news_df = get_crypto_news_cointelegraph(symbol, max_news=100)
    else:
        news_df = pd.DataFrame()

    if news_df.empty:
        df["vader"] = 0.0
        df["finbert"] = 0.0
        df["sentiment_ratio"] = 0.0
        df["sentiment_combined"] = 0.0
        return df

    # Calcular sentimientos
    news_df["text"] = news_df["title"] + ". " + news_df["description"]
    news_df["vader"] = news_df["text"].apply(calcular_sentiment_vader)
    news_df["finbert"] = news_df["text"].apply(calcular_sentiment_finbert)

    # Procesar fechas
    news_df["date"] = pd.to_datetime(news_df["date"], errors="coerce", utc=True)
    news_df.dropna(subset=["date"], inplace=True)
    news_df["date_only"] = news_df["date"].dt.date

    # Asegurar timestamp en df
    if "timestamp" not in df.columns and "datetime" in df.columns:
        df.rename(columns={"datetime": "timestamp"}, inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    df["date_only"] = df["timestamp"].dt.date

    # Agrupar sentimiento por día
    daily_sentiment = news_df.groupby("date_only")[["vader", "finbert"]].mean().reset_index()

    # Hacer merge con df
    df = df.merge(daily_sentiment, on="date_only", how="left")
    df[["vader", "finbert"]] = df[["vader", "finbert"]].fillna(0.0)

    # Calcular ratio y score combinado
    epsilon = 1e-6
    df["sentiment_ratio"] = df["finbert"] / (abs(df["vader"]) + epsilon)
    df["sentiment_combined"] = 0.7 * df["finbert"] + 0.3 * df["vader"]

    # Limpiar
    df.drop(columns=["date_only"], inplace=True)

    return df



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#Delta Managament
# --- Función para cargar df existente ---
def cargar_df_existente(symbol):
    filename = f"{symbol}_datos.txt"
    if os.path.exists(filename):
        df = pd.read_csv(filename, sep="\t")
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
        existing_dates = set(df["timestamp"].dt.date)
    else:
        df = pd.DataFrame()
        existing_dates = set()
    return df, existing_dates

# --- Función para descargar datos según fuente ---
def descargar_datos(symbol, source, granularity, period, limit):
    if source == "bitget":
        return descargar_datos_bitget(symbol, granularity, limit)
    elif source == "yfinance":
        return descargar_datos_yf(symbol, granularity, period, limit)
    else:
        print(f"Fuente no soportada: {source}")
        return None

# --- Función para filtrar solo nuevas fechas ---
def filtrar_nuevas_fechas(df_new, existing_dates):
    df_new["timestamp"] = pd.to_datetime(df_new["timestamp"], utc=True)
    df_new["date_only"] = df_new["timestamp"].dt.date
    df_to_add = df_new[~df_new["date_only"].isin(existing_dates)].copy()
    df_to_add.drop(columns=["date_only"], inplace=True)
    return df_to_add

# --- Función principal para procesar un activo ---
def procesar_activo(symbol, source, granularity, period, limit):
    df_existing, existing_dates = cargar_df_existente(symbol)
    df_new = descargar_datos(symbol, source, granularity, period, limit)
    if df_new is None or df_new.empty:
        print(f"No se descargaron datos para {symbol}")
        return
    df_to_add = filtrar_nuevas_fechas(df_new, existing_dates)
    if df_to_add.empty:
        print(f"No hay nuevas fechas para {symbol}")
        return
    df_to_add = calcular_features(df_to_add)
    df_to_add = agregar_sentiment_diario(df_to_add, symbol, source)
    df_final = pd.concat([df_existing, df_to_add], ignore_index=True) if not df_existing.empty else df_to_add
    save_df(df_final, symbol)



In [ ]:
# --- Loop principal ---
for symbol, source in activos.items():
    procesar_activo(symbol, source, granularity, period, limit)

Descargando desde Bitget: BTCUSDT...
No hay nuevas fechas para BTCUSDT
Descargando desde Bitget: ETHUSDT...
No hay nuevas fechas para ETHUSDT
Descargando desde Yahoo Finance: AAPL...


/tmp/ipython-input-2106467123.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=symbol, interval=interval, period=period)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2106467123.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=symbol, interval=interval, period=period)
[*********************100%***********************]  1 of 1 completed

No hay nuevas fechas para AAPL
Descargando desde Yahoo Finance: MSFT...
No hay nuevas fechas para MSFT
